#Tryout #3
Dropped learning data size to 10000 each class

In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
#from torch.autograd import Function
from torchvision import transforms, datasets
# import math

import torch.nn.functional as F
# import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
# import copy
import tqdm
from PIL import Image

# Load the data (extract)
import os
import zipfile
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
if not (os.path.exists('tmp/train')):
  local_zip = '/content/drive/My Drive/datasets/dogscats/train.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()
  local_zip = '/content/drive/My Drive/datasets/dogscats/test.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

base_dir = '/tmp'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test1')

train_fnames = os.listdir(train_dir)
validation_fnames = os.listdir(validation_dir)

train_cat_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('cat')][:-1500]
train_dog_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('dog')][:-1500]
validation_cat_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('cat')][-500:]
validation_dog_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('dog')][-500:]

print('total training cat images:', len(train_cat_fnames))
print('total training dog images:', len(train_dog_fnames))
print('total validation cat images:', len(validation_cat_fnames))
print('total validation dog images:', len(validation_dog_fnames))

total training cat images: 11000
total training dog images: 11000
total validation cat images: 500
total validation dog images: 500


In [0]:
class CatDogDataset(Dataset):
    def __init__(self, file_list, label=-1, mode='train', transform = None):
        self.file_list = file_list
        self.label = label
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_list[idx])
        if self.transform:
            img = self.transform(img)
            
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.label #file_list[idx]
        
data_transform = transforms.Compose([
    transforms.Resize((45,45)),
    transforms.Resize((450,450)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

train_cats_ds = CatDogDataset(train_cat_fnames, 0, transform = data_transform)
train_dogs_ds = CatDogDataset(train_dog_fnames, 1, transform = data_transform)

train_catdogs_ds = ConcatDataset([train_cats_ds, train_dogs_ds])
dataloader = DataLoader(train_catdogs_ds, batch_size = 32, shuffle=True, num_workers=4)

validation_cats = CatDogDataset(validation_cat_fnames, 0, transform = data_transform)
validation_dogs = CatDogDataset(validation_dog_fnames, 1, transform = data_transform)

validation_catdogs = ConcatDataset([validation_cats, validation_dogs])
validation_size = len(validation_cat_fnames) + len(validation_dog_fnames)
validation_dataloader = DataLoader(validation_catdogs, batch_size = int(validation_size / 100), shuffle=False, num_workers=4)

In [0]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
      for data, target in validation_dataloader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
          pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(validation_dataloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(validation_dataloader.dataset),
      100. * correct / len(validation_dataloader.dataset)))
  return 100. * correct / len(validation_dataloader.dataset)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    addition = 20

    self.cl1 = nn.Conv2d(1,addition,4,1,padding=1)
    self.cl2 = nn.Conv2d(addition,addition*2, 4,1,padding=1)
    addition *=2
    self.cl3 = nn.Conv2d(addition,addition*2, 4,1,padding=1)
    addition *=2
    self.cl4 = nn.Conv2d(addition,addition*2, 4,1,padding=1)
    addition *=2
    self.cl5 = nn.Conv2d(addition,addition*2, 4,1,padding=1) #torch.Size([32, 140, 13, 13]) end
    addition *=2
    self.fc1 = nn.Linear( addition * (13**2), 500)
    self.fc2 = nn.Linear(500, 10)
  def forward(self, x): 
    x = F.relu(self.cl1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.cl2(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.cl3(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.cl4(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.cl5(x))
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, self.fc1.in_features)
    x = F.relu(self.fc1(x)) # new size is batch x 500
    x = self.fc2(x) # new size is batch x 10
    x = F.log_softmax(x, dim=1) # used dim=1 to work on the data not on the batch
    return x
model = Net().to(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

In [11]:
import gc
gc.collect()

41

In [0]:
epochs = 20
itr = 1
p_itr = 10
total_loss = 0
loss_list = []
acc_list = []
for epoch in range(epochs):
    gc.collect()
    model.train()
    for samples, labels in dataloader:
        samples, labels = samples.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(samples)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step()
        
        if itr%p_itr == 0:
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            acc = torch.mean(correct.float())
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, acc))
            loss_list.append(total_loss/p_itr)
            acc_list.append(acc)
            total_loss = 0
            
        itr += 1
    acc = test()
    torch.save(model.state_dict(),"/content/drive/My Drive/datasets/dogscats/cnn3.pt")
    if (acc > 85):
      break
    for param_group in optimizer.param_groups:
      param_group['lr'] = param_group['lr'] * 0.95
print(itr)


[Epoch 1/20] Iteration 10 -> Train Loss: 2.9427, Accuracy: 0.500
[Epoch 1/20] Iteration 20 -> Train Loss: 0.7551, Accuracy: 0.312
[Epoch 1/20] Iteration 30 -> Train Loss: 0.7262, Accuracy: 0.406
[Epoch 1/20] Iteration 40 -> Train Loss: 0.6947, Accuracy: 0.562
[Epoch 1/20] Iteration 50 -> Train Loss: 0.7104, Accuracy: 0.531
[Epoch 1/20] Iteration 60 -> Train Loss: 0.6896, Accuracy: 0.500
[Epoch 1/20] Iteration 70 -> Train Loss: 0.7000, Accuracy: 0.531
[Epoch 1/20] Iteration 80 -> Train Loss: 0.7015, Accuracy: 0.688
[Epoch 1/20] Iteration 90 -> Train Loss: 0.7026, Accuracy: 0.469
[Epoch 1/20] Iteration 100 -> Train Loss: 0.7131, Accuracy: 0.438
[Epoch 1/20] Iteration 110 -> Train Loss: 0.6995, Accuracy: 0.562
[Epoch 1/20] Iteration 120 -> Train Loss: 0.6969, Accuracy: 0.469
[Epoch 1/20] Iteration 130 -> Train Loss: 0.6974, Accuracy: 0.406
[Epoch 1/20] Iteration 140 -> Train Loss: 0.6980, Accuracy: 0.531
[Epoch 1/20] Iteration 150 -> Train Loss: 0.7017, Accuracy: 0.312
[Epoch 1/20] Iterat

In [0]:
criterion = nn.CrossEntropyLoss()